In [1]:
import pandas as pd
import numpy as np
import rdflib
from rdflib.namespace import RDF
from rdflib import Namespace
from rdflib import URIRef, Literal

In [2]:
global g9
g9=rdflib.Graph()
g9.parse('C:/Users/SESA456791/Desktop/Ontology-Bh/Files/Outputs/Graph_State_District_Code_8.nt',format='nt')
print(len(g9))

104476


In [3]:
#************Change Path to files************
judiciary_data=pd.read_csv("C:/Users/SESA456791/Desktop/Ontology-Bh/Files/judiciary_new.csv",header=None) 
judiciary_data=judiciary_data.replace(np.nan,"",regex=True)

In [4]:
judiciary_data.columns=['Cat1','Cat2','Org1','Org2']
judiciary_data.head(5)

,Cat1,Cat2,Org1,Org2
0,Judiciary,Supreme Court of India,['Case Status - pending and disposed cases by ...,
1,Judiciary,Supreme Court of India,['Display Board Supreme Court of India' 'http:...,
2,Judiciary,Supreme Court of India,['Indian Law Institute (ILI) Delhi' 'http://ww...,
3,Judiciary,Supreme Court of India,['Supreme Court of India' 'http://supremecourt...,['Case Status and Daily Orders of Supreme Cour...
4,Judiciary,Supreme Court of India,['Supreme Court of India' 'http://supremecourt...,['Judgment Information system of Supreme Court...


In [5]:
#Function:Changing plural to singular
def change_cat(col_nam,df):
    singular_plural_data=pd.read_csv("C:/Users/SESA456791/Desktop/Ontology-Bh/Files/Singular_Plural.csv") 
    for i in range(singular_plural_data.shape[0]):
        val=singular_plural_data.iloc[i,0]
        new_val=singular_plural_data.iloc[i,1]
        df.loc[(df[col_nam]).apply(lambda x: x.strip())==val,col_nam]=new_val
    return(df)

In [6]:
#Function Call:Changing plural to singular
judiciary_data=change_cat('Cat1',judiciary_data)
judiciary_data=change_cat('Cat2',judiciary_data)

In [7]:
global org
global bhorg
global skos
global bhcatres
global bhorgres

org=Namespace("http://www.w3.org/ns/org#")
bhorg=Namespace("http://semanticwebindia.in/bharathi/ont/org/")
skos=Namespace("http://www.w3.org/2004/02/skos/core#")
bhcatres=Namespace("http://semanticwebindia.in/bharathi/res/cat/")
bhorgres=Namespace("http://semanticwebindia.in/bharathi/res/org/")

In [8]:
pd.set_option('display.max_colwidth', -1)#housekeeping

In [9]:
#finding n_org max
global n_org
l_lit=[]
for s,p,o in g9.triples((None, RDF.type, bhorg.Organisation)):
    str_s_uri=str(s)
    pos=str_s_uri.rfind('/')
    num=int(str_s_uri[pos+1:len(str_s_uri)])
    num_s=str(num)
    num_s=num_s[1:]
    num_i=int(num_s)
    l_lit.append(num_i) #list of n_org identifier values
#print(len(l_lit))
l_lit.sort() #sorting the list
n_org=l_lit[-1] #the current maximum value of n_org

#finding n_cat max, 
global n_cat
n_cat=0
cat_list=[]
for s,p,o in g9.triples((None, skos.prefLabel, None)):
    str_s_uri=str(s)
    pos=str_s_uri.rfind('/')
    if str_s_uri[pos+1:len(str_s_uri)]!='topOT':
        num=int(str_s_uri[pos+1:len(str_s_uri)])
        num_s=str(num)
        num_s=num_s[1:]
        num_i=int(num_s)
        cat_list.append(num_i) #list of n_org identifier values
cat_list.sort() #sorting the list
n_cat=cat_list[-1] #the current maximum value of n_org

In [10]:
print(n_cat)
print(n_org)

47
11102


In [11]:
#create fetch function for cat_id
def fetch_n_cat(cat):
    global n_cat
    global g9
    cat_id=""
    for s,p,o in g9.triples((None, skos.prefLabel,Literal(cat))):
        cat_id=s
    if cat_id=="":
        bhcatres="http://semanticwebindia.in/bharathi/res/cat/"
        n_cat+=1
        cat_id=URIRef(bhcatres+str('2')+str(n_cat))
        g9.add((cat_id,RDF.type,skos.Concept))
        g9.add((cat_id,skos.prefLabel, Literal(cat)))
        g9.add((cat_id,skos.altLabel, Literal(cat)))
        g9.add((cat_id,skos.inScheme,URIRef(bhcatres+'OrganisationType')))
    return(cat_id)

#create fetch function for org_id
def fetch_n_org(org_nam,org_url="",org_abb=""):
    global g9
    global n_org
    org_id=""
    for s,p,o in g9.triples((None, bhorg.hasName,Literal(org_nam))):
        org_id=s
    if org_id=="":
        bhorgres="http://semanticwebindia.in/bharathi/res/org/"
        n_org+=1
        org_id=URIRef(bhorgres+str('1')+str(n_org))
        g9.add((org_id, bhorg.hasName, Literal(org_nam)))
        g9.add((org_id, bhorg.hasAlternateName, Literal(org_nam)))
        if org_url!="":
            g9.add((org_id, bhorg.hasURL, Literal(org_url)))
        if org_abb!="":
            g9.add((org_id, bhorg.hasAbbreviation, Literal(org_abb)))
        g9.add((org_id,RDF.type,bhorg.Organisation))
        g9.add((org_id,org.identifier,Literal(n_org)))
        g9.add((org_id,org.hasOrganizationType,URIRef(bhcatres+str('21')))) #skos(Government) :bhcatres/21
    return(org_id)

In [12]:
#create fetch_cat_broader for broader relation
def fetch_cat_broader(cat1_id,cat2_id):
    global g9
    if not (cat2_id, skos.broader,cat1_id) in g9:
        g9.add((cat2_id, skos.broader,cat1_id))
    if not (cat1_id, skos.broaderOf,cat2_id) in g9:
        g9.add((cat1_id, skos.broaderOf,cat2_id))
        
#create fetch_org_subOrg for sub-organization relation
def fetch_org_subOrg(org1_id,org2_id):
    global g9
    if not (org1_id,org.hasSubOrganization,org2_id) in g9:
        g9.add((org1_id,org.hasSubOrganization,org2_id))
    if not (org2_id,org.subOrganizationOf,org1_id) in g9:
        g9.add((org2_id,org.subOrganizationOf,org1_id))

In [13]:
def org_scrub(org_n):
    l_scrub=[]
    val=org_n
    val=str(val)
    val=val.replace('[',"")
    val=val.replace(']',"")
    if len(val)==0:
        return(l_scrub)
    #print(val)
    pos_url=val.find('http')
    #print(pos_url)
    if pos_url!=-1:
        val_1=val[:pos_url]+'|'+val[pos_url:]
        pos_end_url=val_1[pos_url+1:].find("'")
        val_2=val_1[:pos_url+1+pos_end_url]+'|'+val_1[pos_url+1+pos_end_url:]
    else:
        sch_ch=val[0]
        pos_end=val[1:].find(sch_ch)
        #print(val[:pos_end+2])
        val_2=val[:pos_end+2]+"|"+val[pos_end+2:]
        #print(val_2)
    org_ls=val_2.split('|')
    
    for i in range(len(org_ls)):
        ele=org_ls[i]
        ele_ls=list(ele)
        clean_c=1
        while clean_c>0 and len(ele_ls)>0:
            if ele_ls[0] in ["'",'"',',',' '] and len(ele_ls)>0:
                ele_ls.pop(0)
                #print(ele_ls)
            if len(ele_ls)>0:
                if ele_ls[0] in ["'",'"',',',' ']:
                    clean_c=1
                else:
                    clean_c=0
            else:
                clean_c=0
        clean_c=1
        while clean_c>0 and len(ele_ls)>0:
            if ele_ls[-1] in ["'",'"',',',' ']:
                ele_ls.pop(-1)
            if len(ele_ls)>0:
                if ele_ls[-1] in ["'",'"',',',' ']:
                    clean_c=1
                else:
                    clean_c=0
            else:
                clean_c=0
        ele_j=''.join(ele_ls)
        l_scrub.append(ele_j)
    return(l_scrub)

In [14]:
#Iteration over legislature_data rows
for i in range(judiciary_data.shape[0]):
    
    cat1=judiciary_data.iloc[i,0].strip()
    cat2=judiciary_data.iloc[i,1].strip()
    org1=judiciary_data.iloc[i,2]
    org2=judiciary_data.iloc[i,3]
        
    #check if cat1_id already exists, else create
    cat1_id=fetch_n_cat(cat1)
    
    #check if ((cat1 broader cat21)), ((cat21 broaderOf cat1)) exist, else create
    cat_govt=URIRef(bhcatres+'21')
    fetch_cat_broader(cat_govt,cat1_id)

    #check if cat2_id already exists, else create
    cat2_id=fetch_n_cat(cat2)
    
    #check if ((cat1 broader cat2)), ((cat2 broaderOf cat2)) exist, else create
    fetch_cat_broader(cat1_id,cat2_id)
    
    #check if org1_id already exists, else create
    ##clean and return list
    org1_ls=org_scrub(org1)
    if len(org1_ls)==1:
        org_nam=org1_ls[0]
#     elif len(org1_ls)==2:
#         org_nam=org1_ls[0]
#         org_url=org1_ls[1]
    elif len(org1_ls)==2:
        org_nam=org1_ls[0]
        if org1_ls[1].find('http')>-1:
            org_url=org1_ls[1]
        else:
            org_abb=org1_ls[1]
    else:
        org_nam=org1_ls[0]
        org_url=org1_ls[1]
        org_abb=org1_ls[2]
        
    org1_id=fetch_n_org(org_nam,org_url,org_abb)
    
    #create tuple org1, org:classification, cat2_id
    g9.add((org1_id,org.classification,cat2_id)) 
    #check if org2_id already exists, else create
    org2_ls=org_scrub(org2)
    if (len(org2_ls))>0: 
        if len(org2_ls)==1:
            org_nam=org2_ls[0]
#         elif len(org2_ls)==2:
#             org_nam=org2_ls[0]
#             org_url=org2_ls[1]
        elif len(org2_ls)==2:
            org_nam=org2_ls[0]
            if org2_ls[1].find('http')>-1:
                org_url=org2_ls[1]
            else:
                org_abb=org2_ls[1]
        else:
            org_nam=org2_ls[0]
            org_url=org2_ls[1]
            org_abb=org2_ls[2]
        org2_id=fetch_n_org(org_nam,org_url,org_abb)
    #create tuples for org1-->subOrg-->org2
        fetch_org_subOrg(org1_id,org2_id)
    #create tuple org2, org:classificatoin, cat2_id
#         g9.add((org2_id,org.classification,cat2_id))
        

In [15]:
print(judiciary_data.iloc[71,:])

Cat1    Judiciary                                                                                                    
Cat2    District Court                                                                                               
Org1    ['Bongaigaon District Judiciary (District Court) Assam' 'http://bongaigaonjudiciary.gov.in' 'District Court']
Org2                                                                                                                 
Name: 71, dtype: object


In [16]:
for j in range(judiciary_data.shape[0]): #judiciary_data.shape[0]
#     print(j)
    cat2=judiciary_data.iloc[j,1].strip()
    org2=judiciary_data.iloc[j,3]
    org2_ls=org_scrub(org2)
    if (len(org2_ls))>0: 
        if len(org2_ls)==1:
            org_nam=org2_ls[0]
        elif len(org2_ls)==2:
            org_nam=org2_ls[0]
        else:
            org_nam=org2_ls[0]
            
#     print(cat2,org2,org_nam) # 
    cat2_id=""
    for s,p,o in g9.triples((None,bhorg.hasName,Literal(org_nam))):
#         print(s,p,o)
        for s1,p1,o1 in g9.triples((s,org.classification,None)):
            cat2_id=o1
        if cat2_id=="":
            cat2_id=fetch_n_cat(cat2)
#             print(cat2_id)
            g9.add((s,org.classification,cat2_id))

In [17]:
len(g9)

105784

In [18]:
g9.serialize(format='nt', destination='C:/Users/SESA456791/Desktop/Ontology-Bh/Files/Outputs/Graph_Judiciary_9.nt')

In [19]:
u=Literal("District and Subordinate Judiciary of Southgoa")
for s,p,o in g9.triples((None,bhorg.hasName,u)):
    print(s,p,o)
    for s1,p1,o1 in g9.triples((s,org.classification,None)):
        print(s1,p1,o1)
        for s2,p2,o2 in g9.triples((o1,skos.prefLabel,None)):
            print(s2,p2,o2)

http://semanticwebindia.in/bharathi/res/org/111113 http://semanticwebindia.in/bharathi/ont/org/hasName District and Subordinate Judiciary of Southgoa
http://semanticwebindia.in/bharathi/res/org/111113 http://www.w3.org/ns/org#classification http://semanticwebindia.in/bharathi/res/cat/251
http://semanticwebindia.in/bharathi/res/cat/251 http://www.w3.org/2004/02/skos/core#prefLabel District Court


In [20]:
u=Literal("District and Subordinate Judiciary of Southgoa")
for s,p,o in g9.triples((None,bhorg.hasName,u)):
    for s1,p1,o1 in g9.triples((s,org.subOrganizationOf,None)):
        for s2,p2,o2 in g9.triples((o1,bhorg.hasName,None)):
            print(s2,p2,o2)

http://semanticwebindia.in/bharathi/res/org/111112 http://semanticwebindia.in/bharathi/ont/org/hasName Bombay High Court
